## Creating a Chat Application with MLflow and OpenAI API

![llm_chatbot_model_serving](./Assets/llm_chatbot_model_serving.png)

### Installing Utilities and Libraries

In [ ]:
%pip install openai

### Restarting our Python Environment

In [ ]:
dbutils.library.restartPython()

### Creating the OpenAI Client

In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key = "YOUR_DATABRICKS_ACCESS_TOKEN",
    base_url = "YOUR_DATABRICKS_WORKSPACE_HOSTNAME/serving-endpoints"
)

### Sending a Simple Chat Request

In [ ]:
# OpenAI Request
completion = client.chat.completions.create(
    model="databricks-claude-sonnet-4-5",
    messages=[
        {
            "role":"system",
            "content": [
                {
                    "type": "text", "text": "You are Batman, the protector of Gotham City"
                }
            ]

        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text", "text": "How's Gotham City doing today?"
                }
            ]
        }
    ]
)

# printing the response
print(completion.choices[0].message.content)

### Generating a custom Python Function using MLflow that can be served as a Mosaic AI Endpoint

In [ ]:
import mlflow
from mlflow import pyfunc

class BasicChatBot(pyfunc.PythonModel):
    def __init__(self, model_name: str):
        self.model_name = model_name
    
    def chatCompletionsAPI(self, user_query):
        openai_client = OpenAI(
            api_key = "YOUR_DATABRICKS_ACCESS_TOKEN",
            base_url = "YOUR_DATABRICKS_WORKSPACE_HOSTNAME/serving-endpoints"
        )

        response = openai_client.chat.completions.create(
            model = self.model_name,
            messages = [
                {
                    "role": "system",
                    "content": [
                        {
                            "type": "text", "text": "You are Batman, the protector of Gotham City"
                        }
                    ]
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text", "text": user_query
                        }
                    ]
                }
            ],
            temperature=0.7
        )

        return response.choices[0].message.content
    
    def predict(self, context, data):
        user_query = data["user_query"].iloc[0]
        gpt_response = self.chatCompletionsAPI(user_query)
        return gpt_response

### Saving our Model

In [ ]:
test_model = BasicChatBot(GPT_model="YOUR_MODEL_NAME") # e.g., "databricks-claude-sonnet-4-5"

In [ ]:
from mlflow.models import infer_signature
import pandas as pd

signature = infer_signature(pd.DataFrame([{"user_query": "how is Gotham City today?"}]))
model_path = "basicchatbot"
mlflow.pyfunc.save_model(path=model_path, python_model=test_model, signature=signature)

### Loading our Saved Model

In [ ]:
# Load our custom model from the local artifact store
loaded_pyfunc_model = mlflow.pyfunc.load_model(model_path)

### Testing our Saved Model

In [ ]:
model_input = pd.DataFrame([{"user_query": "hello how are you?"}])

model_response = loaded_pyfunc_model.predict(model_input)

print(model_response)

### Logging our Saved/Loaded Model

In [ ]:
import mlflow

# Log the model as an artifact
with mlflow.start_run() as run:
    mlflow.log_artifacts(local_dir=model_path, artifact_path="BasicChatBot")
    print(f"Model logged with run ID: {run.info.run_id}")

### Testing the Real-Time Endpoint

In [ ]:
{
  "dataframe_records":[
    {
        "user_query":"What is Joker doing in Gotham City today?"
    }
  ]
}